In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

In [ ]:
import os
print(os.listdir('/content/sample_data'))

In [ ]:
import os
print(os.listdir('/content'))

In [ ]:
data = pd.read_csv('sample_data/Updated_Pushkin_Questions_Answers_801 - Sheet1.csv')

In [ ]:
data.head()

In [ ]:
# Просмотр структуры датасета
data.columns = ['Question', 'Answer']  # Переименование столбцов для удобства
data.head()

In [ ]:
# Проверка на наличие пустых значений
data.dropna(inplace=True)

In [ ]:
# Разделение на тренировочную и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Сохранение выборок для дальнейшей работы
train_file_path = "/content/train_dataset.csv"
test_file_path = "/content/test_dataset.csv"
train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)

In [ ]:
train_file_path, test_file_path

In [ ]:
from sklearn.model_selection import train_test_split

# Разделение на train и test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['Question'], data['Answer'], test_size=0.2, random_state=42
)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Загрузка токенайзера и модели
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Загрузка токенизатора и модели
model_name = "t5-small"  # Вы можете использовать "t5-base" для более мощной модели
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
print(data.columns)

In [ ]:
# Токенизация для Seq2Seq
def tokenize_data(data, tokenizer, max_length=512):
    inputs = ["question: " + q for q in data["Question"].tolist()]  # Убедитесь, что используется правильное имя столбца
    targets = data["Answer"].tolist()  # Убедитесь, что используется правильное имя столбца

    input_encodings = tokenizer(inputs, max_length=max_length, truncation=True, padding=True, return_tensors="pt")
    target_encodings = tokenizer(targets, max_length=max_length, truncation=True, padding=True, return_tensors="pt")

    return input_encodings.input_ids, input_encodings.attention_mask, target_encodings.input_ids

# Токенизация train и test данных
train_input_ids, train_attention_mask, train_labels = tokenize_data(train_data, tokenizer)
test_input_ids, test_attention_mask, test_labels = tokenize_data(test_data, tokenizer)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class Seq2SeqDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

# Создаем датасеты
train_dataset = Seq2SeqDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = Seq2SeqDataset(test_input_ids, test_attention_mask, test_labels)

# Создаем DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
# Загружаем модель
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model.to("cpu")  # Используем CPU вместо GPU

# Настройка оптимизатора
optimizer = AdamW(model.parameters(), lr=5e-5)


In [ ]:

# Импорт оптимизатора из PyTorch
from torch.optim import AdamW

# Настройка оптимизатора
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Создаем DataLoader для train и test данных
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

# Настройка параметров обучения
epochs = 3
device = "cpu"  # Убедитесь, что используется CPU, если нет GPU

# Функция потерь
loss_fn = torch.nn.CrossEntropyLoss()

# Перевод модели в режим обучения
model.train()

In [ ]:
# Основной цикл обучения
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    epoch_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        # Обнуление градиентов
        optimizer.zero_grad()

        # Прямой проход (forward pass)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Обратный проход (backward pass)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch Loss: {epoch_loss:.4f}")

In [ ]:
model.save_pretrained("path_to_model")
tokenizer.save_pretrained("path_to_model")